# Use Credit Risk Analytics Notebook Template

In [1]:
from fosforio import snowflake
import pandas as pd
import datetime as dt

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
import seaborn as sns
import subprocess

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

Matplotlib created a temporary cache directory at /tmp/matplotlib-qv_mv225 because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [4]:
# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt
import numpy as np

In [5]:
# To get snowflake connection object with a default snowflake connection created by the user, if available.
#snowflake.get_connection()

# To get snowflake connection object with a specific connection name
snowflake.get_connection(connection_name="FDC_Banking_FS_SNOWFLAKE")

Exception occurred in getting snowflake connection: 'connectionSources'


In [ ]:
# To read a specific dataset published from a snowflake connection
df_original = snowflake.get_dataframe("ATM_TRANSACTION_MASTER_DATA")
df = snowflake.get_dataframe("ATM_TRANSACTION_MASTER_DATA")

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['HOUR'] = df['DATE'].dt.hour
df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
df['QUARTER'] = df['DATE'].dt.quarter
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year
df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
df['DAYOFMONTH'] = df['DATE'].dt.day
df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [ ]:
df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['STATE']= label_encoder.fit_transform(df['STATE'])

In [ ]:
# how to understand word labels. 
atm_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['ATM_ID']= atm_encoder.fit_transform(df['ATM_ID'])

In [ ]:
df.drop(['DATE','BANK_NAME','TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
    'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH']

In [ ]:
print (df.shape)
df = df[df['DISPENSED_AMOUNT'] > 0]
print (df.shape)

In [ ]:
X = df[features]
y = df['DISPENSED_AMOUNT']

In [ ]:
X.head()

In [ ]:
y

In [ ]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

In [ ]:
print(f'Total # of sample in whole dataset: {len(X)}')
print("*****"*10)
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Shape of X_train: {X_train.shape}')
print("*****"*10)
print(f'Total # of sample in test dataset: {len(X_test)}')
print(f'Shape of X_test: {X_test.shape}')

# Random Forest

In [ ]:
tree = RandomForestRegressor(max_depth=4,max_features=4)

In [ ]:
tree.fit(X_train, y_train)

In [ ]:
predictions = tree.predict(X_test)

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

In [ ]:
tree.feature_importances_

In [ ]:
pd.Series(tree.feature_importances_,index=features).sort_values(ascending=False)

In [ ]:
param_grid = [{"max_depth":[3,4,5, None], "max_features":[3,4,5,6,7]}]

In [ ]:
gs = GridSearchCV(estimator=RandomForestRegressor(random_state=123),param_grid = param_grid,cv=10)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.cv_results_['params']

In [ ]:
gs.cv_results_['rank_test_score']

In [ ]:
gs.best_estimator_

In [ ]:
#Build Tree using Best parameters

tree1 = RandomForestRegressor(max_features=7, random_state=123)
tree1.fit(X_train, y_train)
predictions = tree1.predict(X_test)
y_pred = predictions

In [ ]:
X_train.columns

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

In [ ]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    df = pd.DataFrame(payload_dict,index=[0])
    
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['HOUR'] = df['DATE'].dt.hour
    df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
    df['QUARTER'] = df['DATE'].dt.quarter
    df['MONTH'] = df['DATE'].dt.month
    df['YEAR'] = df['DATE'].dt.year
    df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
    df['DAYOFMONTH'] = df['DATE'].dt.day
    df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
    df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

    df['STATE']= label_encoder.transform(df['STATE'])
    df['ATM_ID']= atm_encoder.transform(df['ATM_ID'])
    
    if 'TOTAL_TRANSACTION_COUNT' in df.columns:
        df.drop(['TOTAL_TRANSACTION_COUNT'] ,axis=1, inplace=True)
    
    if 'DOWNTIME_IN_MINS' in df.columns:
        df.drop(['DOWNTIME_IN_MINS'] ,axis=1, inplace=True)
        
    if 'ATM_MAX_CAPACITY' in df.columns:
        df.drop(['ATM_MAX_CAPACITY'] ,axis=1, inplace=True)

    df.drop(['DATE','BANK_NAME'], axis=1,inplace=True)
    
    features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID','HOUR', 'DAYOFWEEK', 
                'QUARTER', 'MONTH', 'YEAR','DAYOFYEAR', 'DAYOFMONTH']

    data = df[features]

    y_pred = model.predict(data)[0]
    return y_pred

In [ ]:
df_original_1 = df_original.copy()
payload  = df_original.iloc[0].to_dict()
#payload1  = df_original.iloc[0].to_dict()
payload

In [ ]:
print ('{ "payload": ', payload, "}")

In [ ]:
df_original_1.drop(['TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], inplace=True, axis=1)
payload1  = df_original_1.iloc[0].to_dict()

In [ ]:
print ('{ "payload": ', payload1, "}")

In [ ]:
req = requests.Request()
req.json = {"payload":payload1}
#req.json = {"payload":payload}
y_req = req
score(tree, y_req)

In [ ]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(tree, y_req)

In [ ]:
y_prediction = pd.Series(y_pred)

In [ ]:
type(X_train), type(X_test), type(y_train), type(y_test), type(y_pred), type(y_prediction)

In [ ]:
## registering the model in Fosfor.
model_reg = register_model(tree1,
               score, 
               name="ATM_DispenseAMT_RandomForest_Regression", 
               description="ATM Dispense Amount RandomForest Regression",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               init_script="pip install snowflake-ml-python==1.0.11",
               y_true=y_test,
               y_pred=y_prediction,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)